<a href="https://colab.research.google.com/github/jumbokh/ML-Class/blob/main/notebooks/5_3_TripAdvisor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 作業
1. 如果只用title這個欄位來預測消費者滿意或不滿意的正確率會是如何？請特別留意類別0的召回率情況。

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['font.sans-serif'] = ['DFKai-sb'] 
plt.rcParams['axes.unicode_minus'] = False
%config InlineBackend.figure_format = 'retina'
import warnings
warnings.filterwarnings('ignore')

df = pd.read_excel('tripadvisor.xlsx', parse_dates=['date'])
import jieba
# 載入繁體字典
jieba.set_dictionary('dict.txt.big')
df['rating'] = (df['rating'] > 3).map({True:1 , False:0})

def preprocessor(s):
    return ' '.join(jieba.cut(s))

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

with open('stop.text','r', encoding='utf-8') as f:
    stops = f.read()
stops = stops.split('\n')

In [2]:
from sklearn.compose import ColumnTransformer
y = df['rating']
X = df[['title', 'content']]
cv = CountVectorizer(preprocessor=preprocessor,    
    token_pattern='(?u)\\b\\w+\\b',
    stop_words=stops)
data_pl = ColumnTransformer([
    ('title', cv, 'title'),
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model_pl = make_pipeline(data_pl, 
                         MultinomialNB())
model_pl.fit(X_train, y_train)
y_pred = model_pl.predict(X_test)
score = model_pl.score(X_test, y_test)
print('測試集的結果', score.round(3))
print(confusion_matrix(y_test, y_pred))
print('綜合報告')
print(classification_report(y_test, y_pred))

Building prefix dict from /content/dict.txt.big ...
Dumping model to file cache /tmp/jieba.u501edca284da514cb68b53a20324f4e3.cache
Loading model cost 1.699 seconds.
Prefix dict has been built successfully.


測試集的結果 0.83
[[  8  79]
 [  8 417]]
綜合報告
              precision    recall  f1-score   support

           0       0.50      0.09      0.16        87
           1       0.84      0.98      0.91       425

    accuracy                           0.83       512
   macro avg       0.67      0.54      0.53       512
weighted avg       0.78      0.83      0.78       512



2. 請用向下取樣的做法，將整個結果再做一次

In [3]:
X = df[['title', 'content']]
cv = CountVectorizer(preprocessor=preprocessor,    
    token_pattern='(?u)\\b\\w+\\b',
    stop_words=stops)
data_pl = ColumnTransformer([
    ('title', cv, 'title'),
])

from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import make_pipeline

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model_pl = make_pipeline(data_pl, 
                         RandomUnderSampler(),
                         MultinomialNB())
model_pl.fit(X_train, y_train)
y_pred = model_pl.predict(X_test)
score = model_pl.score(X_test, y_test)
print('測試集的結果', score.round(3))
print(confusion_matrix(y_test, y_pred))
print('綜合報告')
print(classification_report(y_test, y_pred))

測試集的結果 0.74
[[ 51  36]
 [ 97 328]]
綜合報告
              precision    recall  f1-score   support

           0       0.34      0.59      0.43        87
           1       0.90      0.77      0.83       425

    accuracy                           0.74       512
   macro avg       0.62      0.68      0.63       512
weighted avg       0.81      0.74      0.76       512

